# Example 

In [ ]:
from scripts.data_management import EPManager
from scripts.util import MyLogger, load_sents
from scripts.translators import translate_document
from os.path import join

logger = MyLogger(logfile='log.jsonl')
main_folder = join('ep', 'gpt')
dm = EPManager()

def translation_loop(target_pairs):
    for pair in target_pairs:
        src_lang, tgt_lang = pair
        src_sents, _ = dm.get_sentence_pairs(src_lang, tgt_lang, num_of_sents=100)
        logger.add_dataset_info(name='ep', num_of_sents=100)
        try:
            translate_document(
                text=src_sents,
                src_lang=src_lang,
                tgt_lang=tgt_lang,
                logger=logger,
                mt_folder=main_folder,
                translator='gpt-4.1'
            )
        except Exception as e:
            logger.log_error(
                error=e,
                src_lang=src_lang,
                tgt_lang=tgt_lang,
                translator='gpt-4.1'
            )
            print(str(e))
            continue
    mt_sents = load_sents(main_folder, src_lang, tgt_lang)
    print(f'{len(mt_sents)} translated from {src_lang} to {tgt_lang}')


In [13]:
some_pairs = [
    ('en', 'de'),
    ('de', 'en'),
    ('el', 'de'),
    ('es', 'en'),
    ('fi', 'fr')
]

In [ ]:
translation_loop(some_pairs)